<a href="https://colab.research.google.com/github/Park-da-in/ComputerVision_Seminar/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CV_seminar_project

/content/drive/MyDrive/CV_seminar_project


In [ ]:
%pwd

'/content/drive/MyDrive/CV_seminar_project'

In [ ]:
%ls

 computer_vision_2주차.ipynb
'CV seminar_week2 과제풀이_실습.ipynb'
'CV seminar_week3_과제.ipynb'
'CV seminar_week3_과제_연습.ipynb'
 dataset.py
 ex1.py
 modeling.ipynb
 model_name.pth
 original/
 __pycache__/
 resnet50.pth
 test/
 testing.ipynb
 train/
 valid/
 week4_과제.ipynb
'week4_실습2_파이토치 데이터셋 클래스커스텀.ipynb'
'week4_실습_파이토치 데이터셋 클래스.ipynb의 사본'


#1. 학습한 가중치 불러오기

In [ ]:
import torch
state_dict = torch.load('resnet50.pth')

In [ ]:
#best_epoch = state_dict['epoch']
best_test_acc = state_dict['test_acc']
weights = state_dict['net']

print(f'{10}번째 에포크에서 test셋 기준으로 {best_test_acc}를 달성하였습니다.')

10번째 에포크에서 test셋 기준으로 94.23를 달성하였습니다.


#2. 학습한 가중치를 모델에 적용하기기

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # device 배정
torch.manual_seed(42)
if device == 'cuda':
  torch.cuda.manual_seed_all(42)
device

'cuda'

In [ ]:
from torchvision import models # 모델 라이브러리 함수

resnet_50 = models.resnet50(pretrained=True).to(device) # 선행학습 여부

# finetuning
import torch.nn as nn # 파이토치 뉴럴네트워크 layer 라이브러리
resnet_50.fc = nn.Linear(resnet_50.fc.in_features, 3).to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
# 학습한 가중치 적용완료
resnet_50.load_state_dict(weights)

<All keys matched successfully>

#3. testset의 최종 성능 확인하기
과제 
> 모델은 100epochs를 돌려서 만든 가중치를 이용한다.      
> test_loader를 생성하여, test set의 최종 성능을 평가한다.   
> 텐서보드는 자율로 이용하시오.


In [ ]:
resnet_50.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[ 1.1675e-02,  1.4695e-02, -1.4512e-02,  ..., -4.1412e-02,
                         -4.5293e-02, -7.2762e-02],
                        [ 2.1424e-03,  5.3421e-03,  1.4673e-02,  ...,  1.3904e-04,
                         -2.3815e-02, -4.0930e-02],
                        [ 2.0637e-02,  2.2019e-02,  1.4635e-02,  ...,  1.0040e-01,
                          6.0422e-02,  5.0287e-02],
                        ...,
                        [-1.1446e-03,  2.9083e-02, -9.3956e-03,  ..., -1.2942e-01,
                         -7.8999e-02,  7.4945e-03],
                        [ 4.5069e-03,  4.9802e-02,  6.2837e-02,  ...,  2.3254e-02,
                         -3.4917e-02, -1.5975e-02],
                        [-7.7935e-02, -3.0282e-02, -1.6145e-02,  ...,  3.5838e-02,
                          2.2269e-02,  1.0693e-03]],
              
                       [[-1.7378e-02,  1.3276e-02,  2.5707e-02,  ...,  5.3462e-02,
                          4.1196

In [ ]:
import numpy as np

train_acc_lst = []
train_loss_lst = []

test_acc_lst = []
test_loss_lst = []
epochs = 100

save_dir = '/content/drive/MyDrive/CV_seminar_project'
model_name = 'resnet50'

for epoch in range(1, epochs):
  running_loss = 0.0
  test_running_loss = 0.0

  total = 0
  correct = 0
  
  train_acc = 0
  test_acc = 0

  resnet_50.train()
  for i, (train_img, train_label) in enumerate(train_loader):
    # gpu에 할당
    train_img = train_img.to(device)
    train_label = train_label.to(device)
    optimizer.zero_grad( set_to_none = True ) # 계산했던 가중치 초기화

    output = resnet_50(train_img) # 모델에 입력
    loss = criterion(output, train_label)
    loss.backward() # 미분
    optimizer.step() # 학습

    # loss & acc
    running_loss += loss.item()
    _, predictions = torch.max(output.data ,dim = 1 )
    
    total += train_label.size(0)
    correct += (predictions == train_label).sum().item()
    train_acc += 100 * (correct/total)

  running_loss = round(running_loss/(i+1), 3) # 소수점 반올림
  train_acc = round(train_acc/(i+1), 3)
  
  print(f'Trainset{epoch}/{epochs} Loss : {running_loss}, Accuracy : {train_acc}%')
  train_acc_lst.append(train_acc)
  train_loss_lst.append(running_loss)


  total = 0
  correct = 0
  resnet_50.eval()
  with torch.no_grad():
    for ii, (valid_img, valid_label) in enumerate(valid_loader):
          # gpu에 할당
      valid_img = valid_img.to(device)
      valid_label = valid_label.to(device)

      output = resnet_50(valid_img) # 모델에 입력
      loss = criterion(output, valid_label)

      # loss & acc
      test_running_loss += loss.item()
      _, predictions = torch.max(output.data ,dim = 1 )
      
      total += valid_label.size(0)
      correct += (predictions == valid_label).sum().item()
      test_acc += 100 * (correct/total)

  test_running_loss = round(test_running_loss/(ii+1), 3) # 소수점 반올림
  test_acc = round(test_acc/(ii+1), 3)
  print(f'Validset{epoch}/{epochs} Loss : {test_running_loss}, Accuracy : {test_acc}% \n')
  test_acc_lst.append(test_acc)
  test_loss_lst.append(test_running_loss)

  if np.max(test_acc_lst) <= test_acc: # 현재 에포크의 test_acc가 가장 좋은 성능이라면
    weight = resnet_50.state_dict() # 지금 모델의 가중치를 
    check_point = {
        'net' : weight,

        'test_loss' : test_running_loss,
        'train_loss' : running_loss,

        'train_acc' : train_acc,
        'test_acc' : test_acc,
    }
    
    torch.save(check_point, save_dir + f'/{model_name}.pth')
  

NameError: ignored